<a href="https://colab.research.google.com/github/DonghaeSuh/NLP_Pytorch/blob/main/EDA/Quora_Question_Pairs_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 다운로드 및 불러오기

In [6]:
ls

drive/  sample_data/


In [7]:
cd drive/MyDrive/Pytorch\ NLP/data_in

/content/drive/MyDrive/Pytorch NLP/data_in


In [11]:
cd Quora_Question_Pairs/

/content/drive/MyDrive/Pytorch NLP/data_in/Quora_Question_Pairs


In [12]:
import numpy as np
import pandas as pd
import os
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [13]:
ls

sample_submission.csv.zip  test.csv.zip  train.csv.zip


In [14]:
for file_name in os.listdir():
  zipfile.ZipFile(file_name).extractall()

In [15]:
ls

sample_submission.csv      test.csv      train.csv
sample_submission.csv.zip  test.csv.zip  train.csv.zip


In [16]:
train_df=pd.read_csv('train.csv')
train_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


# 전처리